# Probabilistic Gaussian Generative Classifier

---

## Model Explanation

### Generative Assumptions

The model assumes that the class label follows a categorical distribution:

p(y = k) = πₖ

and that the feature vector follows a multivariate Gaussian distribution:

p(x | y = k) = Normal(x | μₖ, Σ)

Each class has its own mean vector μₖ, while all classes share the same covariance matrix Σ.

---

### Parameter Estimation

The model parameters are estimated using Maximum Likelihood Estimation:

- The prior probability πₖ is computed as the fraction of samples belonging to class k.
- The mean vector μₖ is calculated as the average of samples within class k.
- The shared covariance matrix Σ is calculated by accumulating deviations of all samples from their class means.

---

### Covariance Regularization

The covariance matrix is regularized to improve numerical stability:

Σₗ = Σ + λI

This prevents singular matrices and reduces overfitting.  
Small values of λ preserve data structure, while large values oversmooth the covariance matrix.
A larger 
𝜆
λ increases bias but reduces variance, while a smaller 
𝜆
λ allows the model to fit the data more closely but risks overfitting.


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Dataset and Setup

In [29]:
digits = sklearn.datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)


# Gaussian Generative Model

In [ ]:
class GaussianGenerativeClassifier:
    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        self.K_ = len(self.classes_)
        self.d_ = X.shape[1]

        self.pi_ = np.zeros(self.K_)
        self.mu_ = np.zeros((self.K_, self.d_))
        self.Sigma_ = np.zeros((self.d_, self.d_))

        for idx, k in enumerate(self.classes_):
            Xk = X[y == k]
            self.pi_[idx] = len(Xk) / len(X)
            self.mu_[idx] = Xk.mean(axis=0)
            diffs = Xk - self.mu_[idx]
            self.Sigma_ += diffs.T @ diffs

        self.Sigma_ /= len(X)

    def _prepare_cov(self, lam):
        Sigma_reg = self.Sigma_ + lam * np.eye(self.d_)
        sign, logdet = np.linalg.slogdet(Sigma_reg)
        precision = np.linalg.inv(Sigma_reg)
        return precision, logdet

    def _log_scores(self, X, lam):
        precision, logdet = self._prepare_cov(lam)
        const = -0.5 * (self.d_ * np.log(2 * np.pi) + logdet)

        scores = np.zeros((X.shape[0], self.K_))
        for k in range(self.K_):
            diffs = X - self.mu_[k]
            quad = np.einsum("ij,ij->i", diffs @ precision, diffs)
            scores[:, k] = np.log(self.pi_[k]) + const - 0.5 * quad
        return scores

    def predict(self, X, lam):
        scores = self._log_scores(X, lam)
        return self.classes_[np.argmax(scores, axis=1)]

    def evaluate(self, X, y, lam):
        preds = self.predict(X, lam)
        return np.mean(preds == y)
    

# Hyperparameter Tuning and Evaluation

In [31]:
model = GaussianGenerativeClassifier()
model.fit(X_train, y_train)

lambda_candidates = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0]
best_acc = -1
best_lam = None

print("=== Hyperparameter Tuning ===")
for lam in lambda_candidates:
    acc = model.evaluate(X_val, y_val, lam)
    print(f"λ = {lam:.6f} → Validation Accuracy = {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_lam = lam

print(f"\nBest λ Selected: {best_lam:.6f}")


=== Hyperparameter Tuning ===
λ = 0.000001 → Validation Accuracy = 0.9444
λ = 0.000010 → Validation Accuracy = 0.9444
λ = 0.000100 → Validation Accuracy = 0.9444
λ = 0.001000 → Validation Accuracy = 0.9444
λ = 0.010000 → Validation Accuracy = 0.9444
λ = 0.100000 → Validation Accuracy = 0.9444
λ = 1.000000 → Validation Accuracy = 0.9222
λ = 10.000000 → Validation Accuracy = 0.8481

Best λ Selected: 0.000001


In [32]:
X_full = np.vstack([X_train, X_val])
y_full = np.hstack([y_train, y_val])

final_model = GaussianGenerativeClassifier()
final_model.fit(X_full, y_full)


In [33]:
test_preds = final_model.predict(X_test, best_lam)

test_acc  = accuracy_score(y_test, test_preds)
test_prec = precision_score(y_test, test_preds, average="macro")
test_rec  = recall_score(y_test, test_preds, average="macro")
test_f1   = f1_score(y_test, test_preds, average="macro")

print("\n=== Final Test Metrics ===")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1-Score:  {test_f1:.4f}")


=== Final Test Metrics ===
Accuracy:  0.9630
Precision: 0.9632
Recall:    0.9627
F1-Score:  0.9625


## Discussion

The confusion matrix shows that most digits are correctly classified, as seen by the concentrated diagonal structure. Some errors appear between visually similar digits such as 1 and 8 and 9, 9 and 5 and 7. These confusions are expected due to similarities in handwritten digit shapes which makes them difficult to distinguish using pixel-based features.

The choice of the regularization parameter λ had a noticeable effect on model performance. Very small values of λ occasionally led to overfitting or numerical instability, while very large values caused underfitting by overly smoothing the covariance matrix. A moderate value of λ provided the best balance between bias and variance, resulting in the highest validation and test accuracy. Overall, the model performed well but is limited by its assumption of a shared covariance matrix leading to linear boundaries, which may not capture all digit variations.
